# Second Module
# Optimization Module

# Step Zero

In [1]:
time_periods = list(range(int(input("Enter the start time period (e.g., 0): ").strip()), int(input("Enter the end time period (e.g., 72): ").strip()) + 1))
solar_capital_cost = int(input("Enter the solar installation capital cost for the year of the study (e.g., $1,000,000 for 2021): ").strip())
wind_capital_cost = int(input("Enter the wind installation capital cost for the year of the study (e.g., $1,404,000 for 2021): ").strip())

# Store the variables for use in other scripts
with open('good_model_working/user_inputs.py', 'w') as f:
    f.write(f"time_periods = {time_periods}\n")
    f.write(f"solar_capital_cost = {solar_capital_cost}\n")
    f.write(f"wind_capital_cost = {wind_capital_cost}\n")

# Import Packages

In [2]:
# import all necessary packages & modules
import pyomo.environ as pyomo
import pickle
from good_model_working import utils
import good_model_working
from good_model_working.reload import deep_reload

# Creating Model's Graph

In [3]:
# Creating graph and sets for main module
deep_reload(good_model_working)

# import the model data & sets
input_sets = utils.get_sets('/Users/haniftayarani/good_model/Model Input/all_input_sets_sorted.json')
model_sets = utils.filter_sets(input_sets, utils.gen_to_remove)
graph = utils.create_graph('/Users/haniftayarani/good_model/Model Input/all_input_objects.json')

# Preparing Model's Input

In [4]:
# Selecting the region to run the ED
deep_reload(good_model_working)

user_input = input('Please select your region(s) for analysis (separated by commas): \nAll\nFlorida\nMISO\nNew England\nPJM\nSSP\nSoutheast\nERCOT\nWECC\n')
subgraph, sub_nodes = utils.get_subgraph(user_input, graph)
constraint_to_deactivate = ['policy'] if input("Do you want to include the RFS policy in this study? (yes/no): ").strip().lower() == 'no' else []
rfs_policy_set = None if 'policy' in constraint_to_deactivate else (float(input("Enter the RFS policy percentage (e.g., 30 for 30%): ").strip()) / 100)
# constraint_to_deactivate = ['generator', 'solar', 'wind', 'storage', 'policy']
model_data = {
    'test_nodes': True,
    'test_cons': False,
    'graph': graph, 
    'sets': model_sets, 
    'subgraph_nodes': sub_nodes, 
    'subgraph': subgraph,
    'rfs_policy': rfs_policy_set,
    'constraint_deactivation': constraint_to_deactivate,
}

# Building the Optimization Model

In [5]:
# import good_model_working
deep_reload(good_model_working)

#Change this to point to your version of cbc or use another solver
solver_name='appsi_highs'

# run model 
problem = good_model_working.opt_model.Opt_Model(model_data, solver_name)

[+   5.73] Grid built
[+   0.00] Sets built
[+   0.21] Parameters built
Total demand for each region:
Region: ERC_REST, Total Demand: 2751145.398636087
Region: ERC_WEST, Total Demand: 231734.72559519781
Region: FRCC, Total Demand: 1570909.8735269844
Region: MIS_AMSO, Total Demand: 264529.9431033325
Region: MIS_AR, Total Demand: 308708.07891427266
Region: MIS_D_MS, Total Demand: 181734.51315586868
Region: MIS_IA, Total Demand: 181732.02964569617
Region: MIS_IL, Total Demand: 379503.3340264188
Region: MIS_INKY, Total Demand: 768110.085281914
Region: MIS_LA, Total Demand: 377837.62248327816
Region: MIS_LMI, Total Demand: 780262.0340042331
Region: MIS_MAPP, Total Demand: 73717.73145884342
Region: MIS_MIDA, Total Demand: 249774.84296430898
Region: MIS_MNWI, Total Demand: 788191.6540060375
Region: MIS_MO, Total Demand: 326037.25766873674
Region: MIS_WOTA, Total Demand: 275439.2274530976
Region: MIS_WUMS, Total Demand: 522739.18770741305
Region: NENGREST, Total Demand: 600239.1963267588
Regio

# Run the Optimization Model 

In [6]:
deep_reload(good_model_working.utils)

# Generate simple model statistics
utils.get_model_statistcs(problem.model)
utils.get_total_generator_count(problem.model)

# Solve the model
problem.solve_model()

# print the objective value
print(f"Total system cost: ${pyomo.value(problem.model.obj_func):,.2f}")

 total_variables  total_constraints
          390392             396955
Total generators: 4802.0
Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4635 rows, 366818 cols, 860678 nonzeros
4635 rows, 206112 cols, 538351 nonzeros
4635 rows, 205382 cols, 536891 nonzeros
Presolve : Reductions: rows 4635(-392320); columns 205382(-185010); elements 536891(-891958)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.1715080345e+02 Pr: 4073(4.12867e+10) 0s
      12061     4.7385839011e+08 Pr: 0(0); Du: 0(4.47559e-16) 1s
      12061     4.7385839011e+08 Pr: 0(0); Du: 0(4.47559e-16) 1s
Solving the original LP from the solution after postsolve
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
      12061     4.7386668419e+08 Pr: 13623(121.318); Du: 0(2.56759e-05) 1s
      12061     4.738666

# Saving the Model Output

In [7]:
deep_reload(good_model_working)

# Get model results
results = problem.get_results()
# Specify the path to save the pickle file
pickle_file_path = 'results.pickle'

# Save the dictionary as a pickle file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(results, f)
    
nodes_dict = results.get('nodes', {})